In [2]:
import os
import csv
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    save_circuit
)
from sanchez_ansatz import SanchezAnsatz
from qiskit import transpile
from qiskit_algorithms.optimizers import SPSA
import matplotlib.pyplot as plt
from itertools import product

%matplotlib inline

In [3]:
#qubit_range = [3, 4, 5, 6, 7, 8]
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]

In [ ]:

for num_qubits, eps in product(qubit_range, eps_values):

    print("--"*50)
    print(f"\t\t Experiments for {num_qubits} qubits and {eps} error")
    print("--"*50)


    state = np.random.rand(2**num_qubits)
    state = state / np.linalg.norm(state)

    ansatz = SanchezAnsatz(state, eps)

    ## Running original
    print("--"*50)
    print("\t\t Running ORIGINAL Method")
    print("--"*50)
    init_params = ansatz.init_params
    t_ansatz = transpile(ansatz, basis_gates=["cx", "u"])

    em_original = ExperimentModule(
                    t_ansatz,
                    SPSA(maxiter=1000),
                    target_state=state,
                    init_params=init_params
                )

    result_original = em_original.minimize()

    ## Running modified
    print("--"*50)
    print("\t\t Running MODIFIED method")
    print("--"*50)
    ansatz = SanchezAnsatz(state, eps, build_modified=True)

    init_params = ansatz.init_params
    t_ansatz = transpile(ansatz, basis_gates=["cx", "u"])

    em_modified = ExperimentModule(
                    t_ansatz,
                    SPSA(maxiter=1000),
                    target_state=state,
                    init_params=init_params
                )

    result_modified = em_modified.minimize()

    # Saving comparison plots
    save_plots(em_original, em_modified, f"results/plots/plot_{num_qubits}qb_{eps}eps.pdf")

    # writing csv loss progression
    write_row(em_original._loss_progression, file=f"results/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv")
    write_row(em_modified._loss_progression, file=f"results/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv")

    # writing csv best point
    write_row(em_original.result.x, file=f"results/csv/original_xbest_{num_qubits}qb_{eps}eps.csv")
    write_row(em_modified.result.x, file=f"results/csv/modified_xbest_{num_qubits}qb_{eps}eps.csv")

    # writing op_counts
    write_opcounts(em_original, em_modified, file=f"results/op_counts/counts_{num_qubits}qb_{eps}eps.txt")

    # saving circuits
    save_circuit(em_original._ansatz, file=f"results/circuits/curcuit_{num_qubits}qb_{eps}eps.pkl")